STEP 1: Preprocess Data

In [62]:
import pandas as pd
import numpy as np
np.random.seed(42)
%config IPCompleter.greedy=True

In [63]:
admissions=pd.read_csv('binary.csv')
print admissions.head()
print '\n The length of the file:'+str(len(admissions))

   admit  gre   gpa  rank
0      0  380  3.61     3
1      1  660  3.67     3
2      1  800  4.00     1
3      1  640  3.19     4
4      0  520  2.93     4

 The length of the file:400


In [64]:
data=pd.concat([admissions,pd.get_dummies(admissions['rank'],prefix='rank')],axis=1)
data=data.drop(['rank'],axis=1)
data.head(5)

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,380,3.61,0,0,1,0
1,1,660,3.67,0,0,1,0
2,1,800,4.00,1,0,0,0
3,1,640,3.19,0,0,0,1
4,0,520,2.93,0,0,0,1


In [65]:
# Standarize features
for field in ['gre','gpa']:
    mean,std=data[field].mean(),data[field].std()
    data[field]=(data[field]-mean)/std
data.head(16)

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,-1.798011,0.578348,0,0,1,0
1,1,0.625884,0.736008,0,0,1,0
2,1,1.837832,1.603135,1,0,0,0
3,1,0.452749,-0.525269,0,0,0,1
4,0,-0.586063,-1.208461,0,0,0,1
5,1,1.491561,-1.024525,0,1,0,0
6,1,-0.239793,-1.077078,1,0,0,0
7,0,-1.624876,-0.814312,0,1,0,0
8,1,-0.412928,0.000263,0,0,1,0
9,0,0.972155,1.392922,0,1,0,0


In [66]:
# Split off random 10% of the data for testing

sample=np.random.choice(data.index,size=int(len(data)*0.9),replace=False)
data,test_data=data.loc[sample],data.drop(sample)
print data.head()
print '\n The length of the data is:'+str(len(data))
print
print test_data.head()
print '\n The length of the test data is:'+str(len(test_data))



     admit       gre       gpa  rank_1  rank_2  rank_3  rank_4
209      0 -0.066657  0.289305       0       1       0       0
280      0  0.625884  1.445476       0       1       0       0
33       1  1.837832  1.603135       0       0       1       0
210      0  1.318426 -0.131120       0       0       0       1
93       0 -0.066657 -1.208461       0       1       0       0

 The length of the data is:360

    admit       gre       gpa  rank_1  rank_2  rank_3  rank_4
20      0 -0.759199 -0.577822       0       0       1       0
21      1  0.625884  0.630901       0       1       0       0
48      0 -1.278605 -2.390908       0       0       0       1
50      0  0.452749  1.235263       0       0       1       0
54      0  0.625884 -0.131120       0       0       1       0

 The length of the test data is:40


In [67]:
# Split into features and targets
features,targets=data.drop('admit',axis=1),data['admit']
features_test,targets_test=test_data.drop('admit',axis=1),test_data['admit']
print features.head()
print
print targets_test.head()

          gre       gpa  rank_1  rank_2  rank_3  rank_4
209 -0.066657  0.289305       0       1       0       0
280  0.625884  1.445476       0       1       0       0
33   1.837832  1.603135       0       0       1       0
210  1.318426 -0.131120       0       0       0       1
93  -0.066657 -1.208461       0       1       0       0

20    0
21    1
48    0
50    0
54    0
Name: admit, dtype: int64


In [68]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [69]:
n_records,n_features=features.shape
print n_records,n_features

360 6


In [70]:
epochs=500
n_hidden=3
learnrate=0.5
last_loss=None


In [71]:
weights_input_hidden=np.random.normal(scale=1/n_features**0.5,size=(n_features,n_hidden))                        
print weights_input_hidden
print
weights_hidden_output=np.random.normal(scale=1/n_features**0.5,size=n_hidden)
print weights_hidden_output


[[-0.22097886  0.31713666 -0.01945046]
 [ 0.21175613 -0.15978912  0.17882198]
 [ 0.67923117 -0.30179201 -0.04376196]
 [-0.19849738  0.65033209 -0.17715592]
 [-0.29912034 -0.23608056 -0.05540639]
 [-0.32394707 -0.15041908  0.06417739]]

[-0.04714182  0.3395436  -0.13037616]


In [72]:
for e in xrange(epochs):
    del_w_input_hidden=np.zeros(weights_input_hidden.shape)
    del_w_hidden_output=np.zeros(weights_hidden_output.shape)
    for x,y in zip(features.values,targets):
        hidden_input=x.dot(weights_input_hidden)
        hidden_output=sigmoid(hidden_input)
        output=sigmoid(hidden_output.dot(weights_hidden_output))
        # Calculate the network's prediction error
        error=y-output
        # Calculate error delta for the output unit
        output_error_term=error*output*(1-output)
        
        # propagate errors to hidden layer
        # Calculate the hidden layer's contribution to the error
        hidden_error=np.dot(output_error_term,weights_hidden_output)
        # Calculate the error delta for the hidden layer
        hidden_error_term=hidden_error*hidden_output*(1-hidden_output)
        
        # Update the change in weights
        del_w_hidden_output+=output_error_term*hidden_output
        del_w_input_hidden+=hidden_error_term*x[:,None]
    # Update weights
    weights_input_hidden+=learnrate*del_w_input_hidden/(n_records)
    weights_hidden_output+=learnrate*del_w_hidden_output/(n_records)
    if e%(epochs/10)==0:
        hidden_output=sigmoid(np.dot(features,weights_input_hidden))
        out=sigmoid(np.dot(hidden_output,weights_hidden_output))
        loss=np.mean((out-targets)**2)
        if last_loss and last_loss<loss:
            print 'Train loss:',loss,'Warning -Loss Increading'
        else:
            print 'Train loss:', loss
        last_loss=loss
        
        
        

Train loss: 0.256674708747
Train loss: 0.22568565922
Train loss: 0.220649798035
Train loss: 0.218306306495
Train loss: 0.216481363944
Train loss: 0.21491297116
Train loss: 0.213549856347
Train loss: 0.212362704022
Train loss: 0.211324214926
Train loss: 0.21040917765


In [73]:
# Calculate accuracy on test data
hidden=sigmoid(np.dot(features_test,weights_input_hidden))
out=sigmoid(np.dot(hidden,weights_hidden_output))
predictions=out>0.5
accuracy=np.mean(predictions==targets_test)
print 'Prediction accuracy:{:.3f}'.format(accuracy)

Prediction accuracy:0.750
